In [1]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
data_from_dir = 'C:/Users/student/Desktop/dataset'
base_dir = base_dir='C:/Users/student/Desktop/dataset/set'

train_dir='C:/Users/student/Desktop/dataset/set/train'
val_dir='C:/Users/student/Desktop/dataset/set/val'
test_dir='C:/Users/student/Desktop/dataset/set/test'

train_cats_dir=f'C:/Users/student/Desktop/dataset/set/train/cats'
train_dogs_dir='C:/Users/student/Desktop/dataset/set/train/dogs'
val_cats_dir='C:/Users/student/Desktop/dataset/set/val/cats'
val_dogs_dir='C:/Users/student/Desktop/dataset/set/val/dogs'
test_cats_dir='C:/Users/student/Desktop/dataset/set/test'
test_dogs_dir='C:/Users/student/Desktop/dataset/set/test'

In [5]:
from keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale = 1/255,
                             zoom_range= 0.2)
data_gen = ImageDataGenerator(rescale = 1/255)

train_gen = train_data_gen.flow_from_directory(train_dir,
                                        target_size=(150,150),
                                        batch_size=20,
                                        class_mode='categorical')
val_gen = data_gen.flow_from_directory(val_dir,
                                        target_size=(150,150),
                                        batch_size=20,
                                        class_mode='categorical')
test_gen = data_gen.flow_from_directory(test_dir,
                                        target_size=(150,150),
                                        batch_size=20,
                                        class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [6]:
from keras.applications import VGG16

conv_vgg16 =VGG16(weights='imagenet', include_top=False,input_shape=(150,150,3))
conv_vgg16.summary()

W0715 13:31:21.579968 12884 deprecation_wrapper.py:119] From c:\users\student\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 13:31:21.589895 12884 deprecation_wrapper.py:119] From c:\users\student\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 13:31:21.591940 12884 deprecation_wrapper.py:119] From c:\users\student\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 13:31:21.607848 12884 deprecation_wrapper.py:119] From c:\users\student\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is depre

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [7]:
for layer in conv_vgg16.layers:
    layer.trainable = False

In [13]:
from keras import models, layers, optimizers

model = models.Sequential()

model.add(conv_vgg16)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='categorical_crossentropy',
              metrics=['acc'])


              

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_4 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 514       
Total params: 16,812,610
Trainable params: 2,097,922
Non-trainable params: 14,714,688
_________________________________________________________________


In [14]:
history = model.fit_generator(train_gen,
                              steps_per_epoch = 100,
                              epochs = 10,
                              validation_data = val_gen,
                              validation_steps = 50
)

Epoch 1/10
100/100 [==============================] - 99s 991ms/step - loss: 0.6031 - acc: 0.6850 - val_loss: 0.4709 - val_acc: 0.8200
Epoch 2/10
100/100 [==============================] - 98s 981ms/step - loss: 0.4509 - acc: 0.8160 - val_loss: 0.3937 - val_acc: 0.8430
Epoch 3/10
100/100 [==============================] - 98s 978ms/step - loss: 0.3821 - acc: 0.8490 - val_loss: 0.3315 - val_acc: 0.8760
Epoch 4/10
100/100 [==============================] - 97s 973ms/step - loss: 0.3407 - acc: 0.8665 - val_loss: 0.3135 - val_acc: 0.8830
Epoch 5/10
100/100 [==============================] - 98s 980ms/step - loss: 0.3124 - acc: 0.8745 - val_loss: 0.3109 - val_acc: 0.8820
Epoch 6/10
100/100 [==============================] - 97s 968ms/step - loss: 0.2917 - acc: 0.8835 - val_loss: 0.2928 - val_acc: 0.8850
Epoch 7/10
100/100 [==============================] - 97s 969ms/step - loss: 0.2741 - acc: 0.8970 - val_loss: 0.2582 - val_acc: 0.9090
Epoch 8/10
100/100 [==============================] - 9

In [15]:
# fine tuning

for i, layer in enumerate(conv_vgg16.layers):
    print(i, layer.name)

0 input_1
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [16]:
for layer in conv_vgg16.layers[:15]:
    layer.trainable = False
for layer in conv_vgg16.layers[15:]:
    layer.trainable = True

In [17]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-6),
             loss='categorical_crossentropy',
             metrics=['acc'])


In [18]:
history = model.fit_generator(train_gen,
                              steps_per_epoch = 100,
                              epochs = 10,
                              validation_data = val_gen,
                              validation_steps = 50
)

Epoch 1/10
100/100 [==============================] - 119s 1s/step - loss: 0.2281 - acc: 0.9115 - val_loss: 0.2410 - val_acc: 0.8990
Epoch 2/10
100/100 [==============================] - 118s 1s/step - loss: 0.2091 - acc: 0.9265 - val_loss: 0.2246 - val_acc: 0.9170
Epoch 3/10
100/100 [==============================] - 119s 1s/step - loss: 0.1943 - acc: 0.9295 - val_loss: 0.2314 - val_acc: 0.9040
Epoch 4/10
100/100 [==============================] - 119s 1s/step - loss: 0.1869 - acc: 0.9365 - val_loss: 0.2220 - val_acc: 0.9150
Epoch 5/10
100/100 [==============================] - 118s 1s/step - loss: 0.1778 - acc: 0.9305 - val_loss: 0.2457 - val_acc: 0.8980
Epoch 6/10
100/100 [==============================] - 123s 1s/step - loss: 0.1704 - acc: 0.9375 - val_loss: 0.2122 - val_acc: 0.9040
Epoch 7/10
100/100 [==============================] - 120s 1s/step - loss: 0.1614 - acc: 0.9380 - val_loss: 0.2102 - val_acc: 0.9140
Epoch 8/10
100/100 [==============================] - 119s 1s/step - 

In [47]:
from keras import preprocessing
import time
start = time.time()

img = preprocessing.image.load_img('C:/Users/student/Desktop/oyster.jpg', 
                               target_size = (150,150))
img = preprocessing.image.img_to_array(img)
img = np.reshape(img, (1,150,150,3))/255
print(model.predict(img))

print(time.time() - start)


[[0.9598382  0.04016181]]
0.04089546203613281


In [33]:
model.predict_classes(img)

array([0], dtype=int64)